In [434]:
import globus_automate_client
import mdf_toolbox
import json

In [851]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID (from CFDE)
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [161]:
from getpass import getpass
aws_id = getpass("AWS ID: ")
aws_secret = getpass("AWS Secret: ")
aws_region = "us-east-1"

AWS ID: ········
AWS Secret: ········


In [329]:
from getpass import getpass
smtp_user = getpass("SMTP Username: ")
smtp_pass = getpass("SMTP Password: ")
smtp_hostname = "email-smtp.us-east-1.amazonaws.com"

SMTP Username: ········
SMTP Password: ········


In [845]:
# Definition
loop_flow_def = {
    "title": "#name# Action Loop Flow",
    "description": "Perform multiple #name# Actions",
    "visible_to": ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    "runnable_by": ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    #"visible_to": ["public"],
    #"runnable_by": ["public"],
    "definition": {
        "StartAt": "MoveInputs",
        "States": {
            "MoveInputs": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "action_inputs",
                        "arguments": {
                            "action_inputs.$": "$.action_inputs"
                        },
                        "result_path": "action_inputs"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "CheckInputs"
            },
            # Supports zero Actions
#            "CheckInputs": {
#                "Type": "Pass",
#                "Result": {
#                    "loop_finished.=": "`$.action_inputs` == []"
                    # Can't guarantee simpleeval allows list creation,
                    # other option is to compare to always-empty list slice
                    #"loop_finished.=": "`$.action_inputs` == `$.action_inputs`[0:0]"
#                },
#                "ResultPath": "$.loop_finished",
#                "Next": "ChooseLoop"
#            },
            "CheckInputs": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        #"expression": "action_inputs == []",
                        "expression": "action_inputs == action_inputs[0:0]",
                        "arguments": {
                            "action_inputs.$": "$.inputs.details.action_inputs"
                        },
                        "result_path": "loop_finished"
                    }]
                },
                "ResultPath": "$.loop_info",
                "WaitTime": 86400,
                "Next": "ChooseLoop"
            },
            "ChooseLoop": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.loop_info.details.loop_finished",
                    "BooleanEquals": True,
                    "Next": "FinishLoop"
                }],
                "Default": "#nospace_name#Action"
            }, 
            "#nospace_name#Action": {
                "Type": "Action",
                "ActionUrl": "#action_url#",
                "ActionScope": "#action_scope#",
                "ExceptionOnActionFailure": True,
                "InputPath": "$.inputs.details.action_inputs[0]",
                "ResultPath": "$.#nospace_name#Result",
                "WaitTime": 86400,
                "Next": "RemoveInput"
            },
#            "RemoveInput": {
#                "Type": "Pass",
#                "Result": {
#                    "action_inputs.=": "`$.action_inputs`[1:]"
#                },
#                "ResultPath": "$",
#                "Next": "CheckInputs"
#            },
            "RemoveInput": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "action_inputs[1:]",
                        "arguments": {
                            "action_inputs.$": "$.inputs.details.action_inputs"
                        },
                        "result_path": "action_inputs"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "CheckInputs"
            },
            "FinishLoop": {
                "Type": "Pass",
                "End": True
            }
        }
    }
}
# Variables encased in ## for formatting later.
# Can't .format() a JSON string, because objects {} are interpreted as
# placeholders. Workaround: Replace #variable# directly.
variables = ["name", "nospace_name", "action_url", "action_scope"]

In [846]:
# Action-specific info
all_action_info = [
    # Transfer
    {
        "name": "Transfer",
        "nospace_name": "Transfer",
        "action_url": "https://actions.globus.org/transfer/transfer",
        "action_scope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer"
    },
    # Send Email
    {
        "name": "Send Email",
        "nospace_name": "SendEmail",
        "action_url": "https://actions.globus.org/notification/notify",
        "action_scope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/notification_notify"
    }
]

In [848]:
# Deployments
selected_actions = [
    "Transfer",
    "Send Email"
]
deploy_res = {}
for action_info in all_action_info:
    action_name = action_info["name"]
    if action_name not in selected_actions:
        continue

    # Can't .format() a JSON string, because objects {} are interpreted as
    # placeholders. Instead, .replace() #variable#.
    definition_str = json.dumps(loop_flow_def)
    for var_name, val in action_info.items():
        replace_str = "#" + var_name + "#"
        definition_str = definition_str.replace(replace_str, val)
    flow_definition = json.loads(definition_str)

    #print(json.dumps(flow_definition, indent=4, sort_keys=True))
    #print(flow_definition["definition"])

    flow_deploy_res = flows_client.deploy_flow(
        flow_definition=flow_definition["definition"],
        title=flow_definition["title"],
        description=flow_definition["description"],
        visible_to=flow_definition["visible_to"],
        runnable_by=flow_definition["runnable_by"],
    )

    deploy_res[action_name] = {
        "flow_id": flow_deploy_res["id"],
        "flow_scope": flow_deploy_res["globus_auth_scope"],
        "flow_url": flow_deploy_res["action_url"]
    }
    print(action_name + ":")
    print(deploy_res[action_name]["flow_id"])
    print(deploy_res[action_name]["flow_url"])
    print(deploy_res[action_name]["flow_scope"])
    print()

Transfer:
c270840f-ba31-49db-9fd9-cd0aa67d9545
http://flows.automate.globus.org/flows/c270840f-ba31-49db-9fd9-cd0aa67d9545
https://auth.globus.org/scopes/c270840f-ba31-49db-9fd9-cd0aa67d9545/flow_c270840f_ba31_49db_9fd9_cd0aa67d9545

Send Email:
9cfbab58-51e6-4366-b859-8274f92fe6ac
http://flows.automate.globus.org/flows/9cfbab58-51e6-4366-b859-8274f92fe6ac
https://auth.globus.org/scopes/9cfbab58-51e6-4366-b859-8274f92fe6ac/flow_9cfbab58_51e6_4366_b859_8274f92fe6ac



In [263]:
# Test inputs
action_test_inputs = [{
    "name": "Transfer",
    "action_inputs": [{
        # jgaff_laptop to MDF Connect Dev-P
        "destination_endpoint_id": "ca7550ad-55a9-4762-b558-8f2b15049039",
        "label": "flow_test_a",  # optional
        "source_endpoint_id": "ce6d512a-b414-11e7-b0a7-22000a92523b",
        "transfer_items": [{
            "destination_path": "/home/ubuntu/data/deleteme/",
            "source_path": "/home/jgaff/Downloads/connect_demo/",
            "recursive": True
            }]
    }, {
        # MDF Connect Dev-P to NCSA
        "destination_endpoint_id": "82f1b5c6-6e9b-11e5-ba47-22000b92c6ec",
        "label": "flow_test_b",  # optional
        "source_endpoint_id": "ca7550ad-55a9-4762-b558-8f2b15049039",
        "transfer_items": [{
            "destination_path": "/test/deleteme_outcar",
            "source_path": "/home/ubuntu/data/deleteme/Ag2Al.hP2/OUTCAR",
            "recursive": False
        }]
    }]
}, {
    "name": "Send Email",
    "action_inputs": [{
        "body_template": "This is a test email for $name",
        "body_variables": {
            "name": "the MDF-developed Send Multiple Emails Flow"
        },
        "destination": "jgaff@uchicago.edu",
        #"destination": "materialsdatafacility@uchicago.edu",
        "send_credentials": [{
            # "credential_method": "",
            "credential_type": "smtp",
            "credential_value": {
                "hostname": smtp_hostname,
                "username": smtp_user,
                "password": smtp_pass
            }
        }],
        "sender": "materialsdatafacility@uchicago.edu",
        "subject": "[Ignore] Flow Email Testing",
        "__Private_Parameters": ["send_credentials"]
    }, {
        "body_template": "This is a different email, testing $text for $name",
        "body_variables": {
            "name": "the MDF-developed Send Multiple Emails Flow",
            "text": "multiple emails sent at once"
        },
        "destination": "jgaff@uchicago.edu",
        #"destination": "materialsdatafacility@uchicago.edu",
        # "notification_method": "",
        # "notification_priority": "low",
        "send_credentials": [{
            # "credential_method": "",
            #"credential_type": "ses",
            #"credential_value": {
            #    "aws_access_key_id": aws_id,
            #    "aws_secret_access_key": aws_secret,
            #    "region_name": aws_region
            #}
            "credential_type": "smtp",
            "credential_value": {
                "hostname": smtp_hostname,
                "username": smtp_user,
                "password": smtp_pass
            }
        }],
        "sender": "materialsdatafacility@uchicago.edu",
        "subject": "[Ignore] Flow Email Testing 2",
        "__Private_Parameters": ["send_credentials"]
    }]
}]

In [267]:
# Test submissions
selected_actions = [
    "Transfer",
    "Send Email"
]
test_res = {}
for test_input in action_test_inputs:
    action_name = test_input["name"]
    if action_name not in selected_actions:
        continue
    elif action_name not in deploy_res.keys():
        print("Action '{}' not deployed".format(action_name))
        continue

    flow_info = deploy_res[action_name]
    flow_test_res = flows_client.run_flow(flow_info["flow_id"],
                                          flow_info["flow_scope"],
                                          test_input)
    test_res[action_name] = flow_test_res.data
    print(action_name, "Flow test submitted")
print(json.dumps(test_res, indent=4, sort_keys=True))

Send Email Flow test submitted
{
    "Send Email": {
        "action_id": "c6ddd68e-7a91-4cd6-9f00-8ac4c96ae3bd",
        "completion_time": "None",
        "created_by": "urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1",
        "details": {
            "code": "ActionStarted",
            "description": "State MoveInputs of type Action started",
            "details": {
                "input": {
                    "expressions": [
                        {
                            "arguments": {
                                "action_inputs": [
                                    {
                                        "body_template": "This is a test email for $name",
                                        "body_variables": {
                                            "name": "the MDF-developed Send Multiple Emails Flow"
                                        },
                                        "destination": "jgaff@uchicago.edu",
                    

In [273]:
# Check status of tests
for action_name, test_info in test_res.items():
    flow_info = deploy_res[action_name]
    status = flows_client.flow_action_status(flow_info["flow_id"],
                                             flow_info["flow_scope"],
                                             test_info["action_id"]).data
    print(action_name)
    print(flow_info["flow_id"], test_info["action_id"])
    print(json.dumps(status, indent=4, sort_keys=True))
    print()

Send Email
afadd3a9-ad3d-4f93-90d0-66d56d16ff07 c6ddd68e-7a91-4cd6-9f00-8ac4c96ae3bd
{
    "action_id": "c6ddd68e-7a91-4cd6-9f00-8ac4c96ae3bd",
    "completion_time": "2020-06-30T14:22:37.295000+00:00",
    "created_by": "urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1",
    "details": {
        "output": {
            "SendEmailResult": {
                "action_id": "19qA0c8W0MCxH",
                "completion_time": "2020-06-30T14:22:35.917092",
                "creator_id": "urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1",
                "details": "Email sent to jgaff@uchicago.edu via SMTP",
                "release_after": "2592000",
                "start_time": "2020-06-30T14:22:35.731843",
                "status": "SUCCEEDED"
            },
            "action_inputs": [
                {
                    "body_template": "This is a test email for $name",
                    "body_variables": {
                        "name": "the MDF-develope

In [187]:
deploy_res

{'Transfer': {'flow_id': 'a58c218d-3874-412f-831c-f8db5e7432bf',
  'flow_scope': 'https://auth.globus.org/scopes/a58c218d-3874-412f-831c-f8db5e7432bf/flow_a58c218d_3874_412f_831c_f8db5e7432bf',
  'flow_url': 'http://flows.automate.globus.org/flows/a58c218d-3874-412f-831c-f8db5e7432bf'},
 'Send Email': {'flow_id': '0b1873b3-7bc6-4e15-9ea7-df936bbcf790',
  'flow_scope': 'https://auth.globus.org/scopes/0b1873b3-7bc6-4e15-9ea7-df936bbcf790/flow_0b1873b3_7bc6_4e15_9ea7_df936bbcf790',
  'flow_url': 'http://flows.automate.globus.org/flows/0b1873b3-7bc6-4e15-9ea7-df936bbcf790'}}

In [841]:
action = "Send Email"
'''flows_client.flow_action_log(deploy_res[action]["flow_id"],
                             deploy_res[action]["flow_scope"],
                             test_res[action]["action_id"], limit=100).data'''

'flows_client.flow_action_log(deploy_res[action]["flow_id"],\n                             deploy_res[action]["flow_scope"],\n                             test_res[action]["action_id"], limit=100).data'

In [781]:
aws_region = "us-east-1"
# Templates cannot contain newlines
curator_email_template = ("A new submission is available for curation. "
"Please visit this webpage to curate the submission: $landing_page "
"(If this link leads to a 404 Not Found page after one minute, it has already been curated.)")
author_email_template = ("Your submission has been $decision by a curator "
"for the following reason: $reason")
preamble_text = "Please review the submission and accept or reject it using the buttons below. You must also provide a reason for your decision."
text_box_prompt = "Please enter a reason for your decision. This will be stored and provided to the submitter."
flow_permissions = ["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"]  # jgaff@uchicago.edu


In [852]:
curation_flow_def = {
    "definition": {
        "StartAt": "ChooseNotifyCurator",
        "States": {
            "ChooseNotifyCurator": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.curator_emails",
                    "BooleanEquals": False,
                    "Next": "CurateData"
                }],
                "Default": "EmailVarsStart"
            },
            "EmailVarsStart": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "[]",
                        "result_path": "email_inputs"
                    }, {
                        "expression": "dests",
                        "arguments": {
                            "dests.$": "$.curator_emails"
                        },
                        "result_path": "dests"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "EmailVarsLoop"
            },
            "EmailVarsLoop": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "current_inputs + [new_input]",
                        "arguments": {
                            "current_inputs.$": "$.inputs.details.email_inputs",
                            "new_input": {
                                # "body_mimetype": "",
                                "body_template.=": ("`$.curator_template` or '"
                                                    + curator_email_template + "'"),
                                "body_variables": {
                                    "landing_page.=": ("'https://actions.globus.org/weboption/landing_page/' "
                                                       "+ `$._context.action_id`")
                                },
                                "destination.=": "`$.inputs.details.dests`[0]",
                                # "notification_method": "",
                                # "notification_priority": "low",
                                "send_credentials.$": "$.send_credentials",
                                "sender.$": "$.email_sender",
                                "subject": "Curation Task Available",
                                "__Private_Parameters": ["send_credentials"]
                            }
                        },
                        "result_path": "email_inputs"
                    }, {
                        "expression": "dests[1:] if dests[1:] else False",
                        "arguments": {
                            "dests.$": "$.inputs.details.dests"
                        },
                        "result_path": "dests"
                    }]
                },
                "ResultPath": "$.inputs",
                "WaitTime": 86400,
                "Next": "EmailVarsChoice"
            },
            "EmailVarsChoice": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.inputs.details.dests",
                    "BooleanEquals": False,
                    "Next": "SendCuratorEmails"
                }],
                "Default": "EmailVarsLoop"
            },
            "SendCuratorEmails": {
                "Type": "Action",
                "ActionUrl": deploy_res["Send Email"]["flow_url"],
                "ActionScope": deploy_res["Send Email"]["flow_scope"],
                #"ExceptionOnActionFailure": True,
                "Parameters": {
                    "action_inputs.$": "$.inputs.details.email_inputs"
                },
                "ResultPath": "$.CuratorEmailsResult",
                "WaitTime": 86400,
                "Next": "CurateData"
            },
            "CurateData": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/weboption/wait_for_option",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/weboption_wait_for_option",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "landing_page": {
                        # "favicon_url": "",
                        # "header_background": "",
                        # "header_icon_link": "",
                        # "header_icon_url": "",
                        "display_options_as": "button",
                        "header_text.=": "'Curation for Submission ' + `$._context.action_id`",
                        # "page_title": "",
                        "preamble_text.=": ("`$.curation_text` or '"
                                            + preamble_text + "'"),
                        "selectable_by.$": "$.curation_permissions",
                        "include_text_input_form": True,
                        "text_input_form_prompt": text_box_prompt,
                        "url_suffix.$": "$._context.action_id"
                    },
                    "options": [{
                        "completed_message": "You have accepted this submission. Thank you!",
                        "description": "Accept this submission",
                        "name": "accepted",
                        "selectable_by.$": "$.curation_permissions",
                        "url_suffix.=": "`$._context.action_id` + '_accept'"
                    }, {
                        "completed_message": "You have rejected this submission. Thank you!",
                        "description": "Reject this submission",
                        "name": "rejected",
                        "selectable_by.$": "$.curation_permissions",
                        "url_suffix.=": "`$._context.action_id` + '_reject'"
                    }]
                },
                "ResultPath": "$.CurationResult",
                "WaitTime": 86400,
                "Next": "ChooseNotifyAuthor"
            },
            "ChooseNotifyAuthor": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.author_email",
                    "BooleanEquals": False,
                    "Next": "FinishCuration"
                }],
                "Default": "SendAuthorEmail"
            },
            "SendAuthorEmail": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.=": ("`$.author_template` or '"
                                        + author_email_template + "'"),
                    "body_variables": {
                        "curation_task_id.$": "$._context.action_id",
                        "decision.$": "$.CurationResult.details.name",
                        "reason.$": "$.CurationResult.details.parameters.user_input"
                    },
                    "destination.$": "$.author_email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials.$": "$.send_credentials",
                    "sender.$": "$.email_sender",
                    "subject": "Submission Curated",
                    "__Private_Parameters": ["send_credentials"]
                },
                "ResultPath": "$.AuthorEmailResult",
                "WaitTime": 86400,
                "Next": "FinishCuration"
            },
            "FinishCuration": {
                "Type": "Pass",
                "End": True
            }
        }
    },
    "description": ("Run a curation flow with email notification. This Flow emails a curator, creates a "
                    "curation page accessible by the curator(s), and then emails the author the decision."),
    "runnable_by": flow_permissions,
    "synchronous": False,
    "title": "Curation with Notification",
    "types": ["Action", "Choice"],
    "visible_to": flow_permissions
}
input_schema = {
    "curator_emails": "list of str, or False",
    "curator_template": "str or False",  # variables: $landing_page
    "curation_permissions": "list of str",
    "curation_text": "str or False",
    "author_email": "str or False",
    "author_template": "str or False",  # variables: $curation_task_id, $decision, $reason
    "email_sender": "str",
    "send_credentials": [{}]
}

In [853]:
'''curation_flow_deploy_res = flows_client.deploy_flow(
    flow_definition=curation_flow_def["definition"],
    title=curation_flow_def["title"],
    description=curation_flow_def["description"],
    visible_to=curation_flow_def["visible_to"],
    runnable_by=curation_flow_def["runnable_by"],
)
curation_flow_deploy_res'''

'curation_flow_deploy_res = flows_client.deploy_flow(\n    flow_definition=curation_flow_def["definition"],\n    title=curation_flow_def["title"],\n    description=curation_flow_def["description"],\n    visible_to=curation_flow_def["visible_to"],\n    runnable_by=curation_flow_def["runnable_by"],\n)\ncuration_flow_deploy_res'

In [861]:
#curation_flow_id = curation_flow_deploy_res["id"]
#curation_flow_scope = curation_flow_deploy_res["globus_auth_scope"]
curation_flow_id = "a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6"
curation_flow_scope = "https://auth.globus.org/scopes/a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6/flow_a24d39dd_f8b7_4287_ba4d_cdd8e36fcee6"
print(curation_flow_id)
print(curation_flow_deploy_res["action_url"])
print(curation_flow_scope)

a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6
http://flows.automate.globus.org/flows/a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6
https://auth.globus.org/scopes/a24d39dd-f8b7-4287-ba4d-cdd8e36fcee6/flow_a24d39dd_f8b7_4287_ba4d_cdd8e36fcee6


In [888]:
curation_flow_input = {
    #"curator_emails": ["jgaff@uchicago.edu", "jgaff@globus.org"],
    "curator_emails": ["jgaff@uchicago.edu"],
    "curator_template": "Curator Template test: $landing_page",
    "curation_text": "Here is some sample text. https://example.org",
    "author_email": "jgaff@uchicago.edu",
    "author_template": "This is a test email for '$curation_task_id': Decision: $decision, reason: $reason",
    "email_sender": "materialsdatafacility@uchicago.edu",
    "curation_permissions": ["public"],  #["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    "send_credentials": [{
        # "credential_method": "",
        "credential_type": "smtp",
        "credential_value": {
            "hostname": smtp_hostname,
            "username": smtp_user,
            "password": smtp_pass
#        "credential_type": "ses",
#        "credential_value": {
#            "aws_access_key_id": aws_id,
#            "_private_aws_secret_access_key": aws_secret,
#            "region_name": aws_region
        }
    }],
    "__Private_Parameters": ["send_credentials"]
}

In [889]:
curation_flow_res = flows_client.run_flow(curation_flow_id,
                                          curation_flow_scope,
                                          curation_flow_input)
curation_flow_res.data

{'action_id': '8ebd0bdd-1233-48a5-abc0-08362be8282e',
 'completion_time': 'None',
 'created_by': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
 'details': {'code': 'ActionStarted',
  'description': 'State EmailVarsStart of type Action started',
  'details': {'input': {'expressions': [{'expression': '[]',
      'result_path': 'email_inputs'},
     {'arguments': {'dests': ['jgaff@uchicago.edu']},
      'expression': 'dests',
      'result_path': 'dests'}]},
   'state_name': 'EmailVarsStart',
   'state_type': 'Action'},
  'time': '2020-08-19T18:11:27.374000+00:00'},
 'start_time': '2020-08-19T18:11:27.244000+00:00',
 'status': 'ACTIVE'}

In [894]:
flows_client.flow_action_status(curation_flow_id,
                                curation_flow_scope,
                                curation_flow_res["action_id"]).data

{'action_id': '8ebd0bdd-1233-48a5-abc0-08362be8282e',
 'completion_time': '2020-08-19T18:36:22.032000+00:00',
 'created_by': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
 'details': {'output': {'AuthorEmailResult': {'action_id': '1PrUtYZto5RGR',
    'completion_time': '2020-08-19T18:36:21.809785',
    'creator_id': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
    'details': 'Email sent to jgaff@uchicago.edu via SMTP',
    'release_after': '2592000',
    'start_time': '2020-08-19T18:36:21.467716',
    'status': 'SUCCEEDED'},
   'CurationResult': {'action_id': 'wPBZfbflXPbw',
    'completion_time': '2020-08-19 18:34:55.466227+00:00',
    'creator_id': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
    'details': {'body': {'data': ''},
     'name': 'accepted',
     'parameters': {'user_input': 'test'},
     'url_suffix': '8ebd0bdd-1233-48a5-abc0-08362be8282e_accept'},
    'display_status': None,
    'label': None,
    'manage_by': 

In [815]:
flows_client.flow_action_log(curation_flow_id,
                             curation_flow_scope,
                             curation_flow_res["action_id"], limit=100).data

{'entries': [{'action_url': 'https://actions.globus.org/weboption/wait_for_option',
   'code': 'ActionPolled',
   'description': 'Polling for completion of action state CurateData',
   'details': {'state_name': 'CurateData'},
   'time': '2020-07-29T21:13:52.059000+00:00'},
  {'action_url': 'https://actions.globus.org/weboption/wait_for_option',
   'code': 'ActionPolled',
   'description': 'Polling for completion of action state CurateData',
   'details': {'state_name': 'CurateData'},
   'time': '2020-07-29T21:13:36.848000+00:00'},
  {'code': 'ActionCompleted',
   'description': 'State CurateData of type Action completed',
   'details': {'output': {'CuratorEmailsResult': {'action_id': '73b96f46-4467-4dab-8ee8-1f9ab29c6bd3',
      'completion_time': '2020-07-29T21:13:21.311000+00:00',
      'created_by': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
      'details': {'output': {'SendEmailResult': {'action_id': '1N0l4ngvQ1JbD',
         'completion_time': '2020-07-29T21

In [663]:
send_credentials = [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }]

In [708]:
dumb_flow_def = {
    "StartAt": "EmailVarsStart",
    "States": {
        "EmailVarsStart": {
            "Type": "Action",
            "ActionUrl": "https://actions.globus.org/expression_eval",
            "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
            "ExceptionOnActionFailure": True,
            "Parameters": {
                "expressions": [{
                    "expression": "[]",
                    "result_path": "email_inputs"
                }, {
                    "expression": "dests",
                    "arguments": {
                        "dests": ["jgaff@uchicago.edu", "jgaff@globus.org"]
                    },
                    "result_path": "dests"
                }]
            },
            "ResultPath": "$.inputs",
            "WaitTime": 86400,
            "Next": "EmailVarsLoop"
        },
        "EmailVarsLoop": {
            "Type": "Action",
            "ActionUrl": "https://actions.globus.org/expression_eval",
            "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
            "ExceptionOnActionFailure": True,
            "Parameters": {
                "expressions": [{
                    "expression": "current_inputs + [new_input]",
                    "arguments": {
                        "current_inputs.$": "$.inputs.details.email_inputs",
                        "new_input": {
                            # "body_mimetype": "",
                            "body_template.=": ("`$._context.action_id` + '"
                                                + curator_email_template + "'"),
                            "body_variables": {
                                "landing_page.=": ("'https://actions.globus.org/weboption/landing_page/' "
                                                   "+ `$._context.action_id`")
                            },
                            "destination.=": "`$.inputs.details.dests`[0]",
                            # "notification_method": "",
                            # "notification_priority": "low",
                            "send_credentials": send_credentials,
                            "sender": "materialsdatafacility@uchicago.edu",
                            "subject": "Curation Task Available",
                            "__Private_Parameters": ["send_credentials"]
                        }
                    },
                    "result_path": "email_inputs"
                }, {
                    "expression": "dests[1:] if dests[1:] else False",
                    "arguments": {
                        "dests.$": "$.inputs.details.dests"
                    },
                    "result_path": "dests"
                }]
            },
            "ResultPath": "$.inputs",
            "WaitTime": 86400,
            "Next": "EmailVarsChoice"
        },
        "EmailVarsChoice": {
            "Type": "Choice",
            "Choices": [{
                "Variable": "$.inputs.details.dests",
                "BooleanEquals": False,
                "Next": "EndStep"
            }],
            "Default": "EmailVarsLoop"
        },
        "EndStep": {
            "Type": "Action",
            "ActionUrl": "https://actions.globus.org/weboption/wait_for_option",
            "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/weboption_wait_for_option",
            "ExceptionOnActionFailure": True,
            "Parameters": {
                "landing_page": {
                    # "favicon_url": "",
                    # "header_background": "",
                    # "header_icon_link": "",
                    # "header_icon_url": "",
                    "display_options_as": "button",
                    "header_text.=": "'Curation for Submission ' + `$._context.action_id`",
                    # "page_title": "",
                    "preamble_text": preamble_text,
                    "selectable_by": ["public"],
                    "include_text_input_form": True,
                    "text_input_form_prompt": text_box_prompt,
                    "url_suffix.$": "$._context.action_id"
                },
                "options": [{
                    "completed_message": "You have accepted this submission. Thank you!",
                    "description": "Accept this submission",
                    "name": "accepted",
                    "selectable_by": ["public"],
                    "url_suffix.=": "`$._context.action_id` + '_accept'"
                }, {
                    "completed_message": "You have rejected this submission. Thank you!",
                    "description": "Reject this submission",
                    "name": "rejected",
                    "selectable_by": ["public"],
                    "url_suffix.=": "`$._context.action_id` + '_reject'"
                }]
            },
            "ResultPath": "$.CurationResult",
            "WaitTime": 86400,
            "End": True
        }
    }
}

In [839]:
'''
dumb_flow_deploy = flows_client.deploy_flow(
    flow_definition=dumb_flow_def,
    title="Testing Flow",
    visible_to=["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
    runnable_by=["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],
)
dumb_flow_deploy
'''

'\ndumb_flow_deploy = flows_client.deploy_flow(\n    flow_definition=dumb_flow_def,\n    title="Testing Flow",\n    visible_to=["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],\n    runnable_by=["urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1"],\n)\ndumb_flow_deploy\n'

In [710]:
dumb_flow_id = dumb_flow_deploy["id"]
dumb_flow_scope = dumb_flow_deploy["globus_auth_scope"]
print(dumb_flow_id)
print(dumb_flow_scope)

17fb892f-3255-4615-aa59-a336d7cc6bee
https://auth.globus.org/scopes/17fb892f-3255-4615-aa59-a336d7cc6bee/flow_17fb892f_3255_4615_aa59_a336d7cc6bee


In [711]:
dumb_flow_input = {
    # pass
}

In [714]:
dumb_flow_res = flows_client.run_flow(dumb_flow_id,
                                      dumb_flow_scope,
                                      dumb_flow_input)
dumb_flow_res.data

{'action_id': '44f5e909-0c02-4ced-b616-d2e6928292a1',
 'completion_time': 'None',
 'created_by': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
 'details': {'code': 'ActionStarted',
  'description': 'State EmailVarsStart of type Action started',
  'details': {'input': {'expressions': [{'expression': '[]',
      'result_path': 'email_inputs'},
     {'arguments': {'dests': ['jgaff@uchicago.edu', 'jgaff@globus.org']},
      'expression': 'dests',
      'result_path': 'dests'}]},
   'state_name': 'EmailVarsStart',
   'state_type': 'Action'},
  'time': '2020-07-23T14:00:20.883000+00:00'},
 'start_time': '2020-07-23T14:00:20.857000+00:00',
 'status': 'ACTIVE'}

In [722]:
flows_client.flow_action_status(dumb_flow_id,
                                dumb_flow_scope,
                                dumb_flow_res["action_id"]).data

{'action_id': '44f5e909-0c02-4ced-b616-d2e6928292a1',
 'completion_time': 'None',
 'created_by': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
 'details': {'action_url': 'https://actions.globus.org/weboption/wait_for_option',
  'code': 'ActionPolled',
  'description': 'Polling for completion of action state EndStep',
  'details': {'state_name': 'EndStep'},
  'time': '2020-07-23T14:01:17.597000+00:00'},
 'start_time': '2020-07-23T14:00:20.857000+00:00',
 'status': 'ACTIVE'}

In [359]:
flow_permissions

['urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1']

In [829]:
ref_auth = list(mdf_toolbox.login(services="https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/notification_notify").values())[0]
action_client = globus_automate_client.create_action_client("https://actions.globus.org/notification/notify", ref_auth.access_token)


In [833]:
email_template = """A new dataset has been submitted to MDF Connect.
Title: $title
Submitter: $submitter ($email)
Source ID: $source_id
Time submitted: $sub_time
Curation required: $curation
"""
email_template2 = "Test email\nTitle: $title\nSubmitter: $submitter ($email)\n\nEnd"
email_creds = {
    "credential_type": "smtp",
        "credential_value": {
            "hostname": smtp_hostname,
            "username": smtp_user,
            "password": smtp_pass
        }
}
body = {
    # "body_mimetype": "",
    "body_template": email_template2,
    "body_variables": {
        "title": "Email Test",
        "submitter": "JGaff",
        "email": "foo@bar.com",
        "source_id": "foobar_v99.99",
        "sub_time": "26:98PM",
        "curation": str(True)
    },
    #"destination": "materialsdatafacility@uchicago.edu",
    "destination": "jgaff@uchicago.edu",
    "send_credentials": [email_creds],
    "sender": "materialsdatafacility@uchicago.edu",
    "subject": "New MDF Connect Submission",
    #"__Private_Parameters": ["send_credentials"]
}

In [834]:
res = action_client.run(body)

In [836]:
res

GlobusHTTPResponse({'action_id': 'IUW8ChSkMYTi', 'completion_time': '2020-08-04T18:57:01.542699+00:00', 'creator_id': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1', 'details': 'Email sent to jgaff@uchicago.edu via SMTP', 'release_after': 2592000, 'start_time': '2020-08-04T18:57:01.162982+00:00', 'status': 'SUCCEEDED'})

In [838]:
action_client.status(res["action_id"]).data

{'action_id': 'IUW8ChSkMYTi',
 'completion_time': '2020-08-04T18:57:01.542699',
 'creator_id': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1',
 'details': 'Email sent to jgaff@uchicago.edu via SMTP',
 'release_after': '2592000',
 'start_time': '2020-08-04T18:57:01.162982',
 'status': 'SUCCEEDED'}